# Walabot keypoint localization network


In [9]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *

import sys, os, warnings, time, glob

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Activation, Conv2DTranspose

print("keras version {}".format(tf.keras.__version__)); del keras
print("tensorflow version {}".format(tf.__version__))

from utils.tf_records_generator import *
from models.res_net import *

%load_ext autoreload
%autoreload 2

keras version 2.2.4-tf
tensorflow version 1.13.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define data loader

In [10]:
# Define global variables
EPOCHS = 30
SUM_OF_ALL_DATASAMPLES = 200
BATCHSIZE = 8
SHUFFLE_BUFFER = 256

In [11]:
TEST_SET_PATH = "/home/kingkolibri/10_catkin_ws/test_records/"
TRAIN_SET_PATH = "/home/kingkolibri/10_catkin_ws/train_records/"

In [12]:
train_files = glob.glob(TRAIN_SET_PATH + "wlb*.tfrecord")
test_files = glob.glob(TEST_SET_PATH + "wlb_*.tfrecord")
train_files

['/home/kingkolibri/10_catkin_ws/train_records/wlb_20190825_OutsideAudimax.tfrecord']

In [13]:
def create_dataset(filenames, repetitions=-1):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Maps the parser on every filepath in the array
    dataset = dataset.map(parse_wlb_example, num_parallel_calls=8)
    dataset = dataset.repeat(repetitions)     
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCHSIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create tf representation of the iterator
    wlb, heatmaps = iterator.get_next()
    
    return wlb, heatmaps

## Build and train model

In [14]:
from keras.applications.resnet50 import ResNet50

In [15]:
STEPS_PER_EPOCH= int(SUM_OF_ALL_DATASAMPLES / BATCHSIZE)

#Get your datatensors
inputs, labels = create_dataset(train_files)

# Define Input
model_input = tf.keras.layers.Input(tensor=inputs)

In [18]:
#X_input = Input(input_shape)

# Zero-Padding
X = ZeroPadding2D((3, 3))(model_input)

# Stage 1
X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

# Stage 2
X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

### START CODE HERE ###

# Stage 3
X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

# Stage 4
X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

# Stage 5
X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

# Average Pooling
X = AveragePooling2D((2, 2), name='avg_pool')(X)

### END CODE HERE ###

# output layer
X = Flatten()(X)
X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

# Create model
model = Model(inputs=X_input, outputs=X, name='ResNet50')

localizer_model.summary()

NameError: name 'glorot_uniform' is not defined

### Train model

In [ ]:
#Compile model
localizer_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
                        loss='mean_squared_error',
                        metrics=['mean_squared_error', 'acc'],
                        target_tensors=[labels]
                   )

In [ ]:
#Train the model
train_hist = localizer_model.fit(epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH)

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
ax1.plot(train_hist.history['loss'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)  # we already handled the x-label with ax1
ax2.plot(train_hist.history['acc'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title("Training history")
plt.show()

### Test trained model

In [ ]:
def create_dataset(filenames, repetitions=-1):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Maps the parser on every filepath in the array
    dataset = dataset.map(parse_p2g_example, num_parallel_calls=8)
    dataset = dataset.repeat(repetitions) # will go on forever        
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCHSIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create tf representation of the iterator
    _, rdm, heatmaps = iterator.get_next()
    
    return rdm, heatmaps

In [ ]:
#Get your datatensors
inputs, labels = create_dataset(test_files, repetitions=1)

#Combine it with keras
model_input = tf.keras.layers.Input(tensor=inputs)

localizer_model.evaluate(
    x=test_set_iterator,
    verbose=1,
)


## Visualize result

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #execute init_op
    sample_rdm = sess.run(inputs)
    sample_heatmaps = sess.run(labels)
    sample_outputs = localizer_model.predict_on_batch(inputs)

In [ ]:
sample_outputs.shape

### Visualize sample heatmaps

In [ ]:
keypoint_columns = [
    'LAnkle', 
    'LElbow', 
    'LHip', 
    'LKnee',
    'LShoulder',
    #['keypoint_0_LSmallToe_x', 'keypoint_0_LSmallToe_y'],
    'LWrist',
    #['keypoint_0_MidHip_x', 'keypoint_0_MidHip_y'],
    #['keypoint_0_Neck_x', 'keypoint_0_Neck_y'],
    'Nose',
    'RAnkle',
    #['keypoint_0_RBigToe_x','keypoint_0_RBigToe_y'], 
    #['keypoint_0_REar_x','keypoint_0_REar_y'], 
    'RElbow', 
    #['keypoint_0_REye_x','keypoint_0_REye_y'], 
    #['keypoint_0_RHeel_x','keypoint_0_RHeel_y'], 
    'RHip', 
    'RKnee', 
    'RShoulder',
    #['keypoint_0_RSmallToe_x', 'keypoint_0_RSmallToe_y'], 
    'RWrist' 
]

### Visualize estimated keypoints

In [ ]:
y_test = sample_outputs
y_pred = sample_heatmaps

In [ ]:
heigth = y_test.shape[1]
width = y_test.shape[2]
num_keypoints = y_test.shape[3]



for i in np.random.randint(y_test.shape[0], size=(2,)):
    
    
    fig = plt.figure(figsize=(3,3))
  #  ax = fig.add_subplot(1,1,1)
  #  ax.imshow(X_train[i,:,:,0], cmap="gray")
  #  ax.axis("off")
    
    fig = plt.figure(figsize=(20,3))
    count = 1
    
    for j, keypoint in enumerate(keypoint_columns):
        ax = fig.add_subplot(2, num_keypoints, count)
        ax.set_title(keypoint)
        ax.axis("off")
        count += 1
        ax.imshow(y_pred[i,:,:,j])
        if j == 0:
            ax.set_ylabel("prediction")
            
    for j, keypoint in enumerate(keypoint_columns):
        ax = fig.add_subplot(2, num_keypoints,count)
        count += 1
        ax.imshow(y_test[i,:,:,j])   
        ax.axis("off")
        if j == 0:
            ax.set_ylabel("true")
    plt.show()

### Visualize stick man

In [ ]:
keypoint_pairs = [
    [('Neck', 'RShoulder'), None],
    [('Neck', 'LShoulder'), None],
    [('RShoulder', 'RElbow'), None],
    [('RElbow', 'RWrist'), None],
    [('LShoulder', 'LElbow'), None],
    [('LElbow', 'LWrist'), None],
    [('Neck', 'RHip'), None],
    [('RHip', 'RKnee'), None],
    [('RKnee', 'RAnkle'), None],
    [('Neck', 'LHip'), None],
    [('LHip', 'LKnee'), None],
    [('LKnee', 'LAnkle'), None],
    [('Neck', 'Nose'), None],
    [('Nose', 'REye'), None],
    [('REye', 'REar'), None],
    [('Nose', 'LEye'), None],
    [('LEye', 'LEar'), None],
    [('RShoulder', 'REar'), None],
    [('LShoulder', 'LEar'), None],
    [('LHip', 'LAnkle'), 'LKnee'],
    [('RHip', 'RAnkle'), 'RKnee'],
    [('RShoulder', 'RAnkle'), 'RElbow'],
    [('LShoulder', 'LAnkle'), 'LElbow'],
    [('RHip', 'LHip'), 'Neck'],
    [('RHip', 'RShoulder'), 'Neck'],
    [('LHip', 'LShoulder'), 'Neck'],
]

In [ ]:
body_pred = {}
body_true = {}
for j, keypoint in enumerate(keypoint_columns):
    x_pred = y_pred[i,:,:,j]
    x_true = y_test[i,:,:,j]
    body_pred[keypoint] = np.unravel_index(y_test[i,:,:,j].argmax(), y_test[i,:,:,j].shape)
    body_true[keypoint] = np.unravel_index(y_pred[i,:,:,j].argmax(), y_pred[i,:,:,j].shape)

Let's first check the visualization of the samples

In [ ]:
body_true

In [ ]:
img = np.zeros((64,64,3), np.uint8)

import cv2
image_h, image_w = img.shape[:2]

centers = {}

# draw point
for key in keypoint_columns:
    if key not in body_true.keys():
        continue

    body_part = body_true[key]
    center = (body_part[1], body_part[0])
    centers[key] = center
    img[center[1], center[0], :] =  [0, 255, 0]

In [ ]:
plt.imshow(img)

In [ ]:
# draw line
for pair_order, pair in enumerate(keypoint_pairs):
    if pair[0][0] not in body_true.keys() or pair[0][1] not in body_true.keys() or pair[1] in body_true.keys():
        continue

    cv2.line(img, centers[pair[0][0]], centers[pair[0][1]], [255, 0, 0], 1)

In [ ]:
plt.imshow(img)

Now lets take a look at the network output

In [ ]:
img = np.zeros((64,64,3), np.uint8)

import cv2
image_h, image_w = img.shape[:2]

centers = {}

# draw point
for key in keypoint_columns:
    if key not in body_pred.keys():
        continue

    body_part = body_pred[key]
    center = (body_part[1], body_part[0])
    centers[key] = center
    img[center[1], center[0], :] =  [0, 255, 0]
    
# draw line
for pair_order, pair in enumerate(keypoint_pairs):
    if pair[0][0] not in body_pred.keys() or pair[0][1] not in body_pred.keys() or pair[1] in body_pred.keys():
        continue

    cv2.line(img, centers[pair[0][0]], centers[pair[0][1]], [255, 0, 0], 1)
    
plt.imshow(img)